In [5]:
#load the modules
import keras
from keras import models, layers
from keras.activations import relu, softmax
from keras.applications import VGG19
from keras.models import Sequential, load_model, Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Dense, Flatten
import matplotlib.pyplot as plt
import sys
from PIL import Image
sys.modules['Image'] = Image

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Aug  3 15:46:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
train_data = ImageDataGenerator(
        rotation_range=40,
        zoom_range=0.2,
        rescale = 1./255)

train_generator = train_data.flow_from_directory(
        directory="/content/drive/MyDrive/Thesis/Detectron_cropped_images/IndividualID/great_tits/Train/",
        target_size=(224, 224),
        batch_size=8,
        shuffle=True)

Found 6370 images belonging to 10 classes.


In [9]:
#defining the validation data generator
val_data = ImageDataGenerator(rescale = 1./255)

val_generator = val_data.flow_from_directory(
        directory="/content/drive/MyDrive/Thesis/Detectron_cropped_images/IndividualID/great_tits/Val/",
        target_size=(224, 224),
        batch_size=8,
        shuffle=True)

Found 988 images belonging to 10 classes.


In [10]:
#load the pre-trained VGG19 from keras
vgg19 = VGG19(input_shape=(224,224,3), weights='imagenet', include_top=False)
x = vgg19.layers[-1].output
#add dropout and the fully connected layer
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
#add a dense layer with a value equal to the number of classes
predictors = Dense(10, activation='softmax')(x)
# Create the model
vgg19model = Model(inputs=vgg19.inputs, outputs=predictors)

80134624/80134624 [==============================] - 5s 0us/step


In [42]:
vgg19model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [11]:
# define where to save the model after each epoch
filepath = "/content/drive/MyDrive/Thesis/Saved_model.h5"
# add a critera to save only if there was an improvement in the model comparing
# to the previous epoch (in this caset the model is saved if there was a decrease in the loss value)
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# stop training if there is no improvement in model for 3 consecutives epochs.
early_stopping_monitor = EarlyStopping(patience=3)
callbacks_list = [checkpoint, early_stopping_monitor]

In [12]:
# Compile the model
vgg19model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5),#define the optimizer and the learning rate
              metrics=['acc'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [46]:
#train the model
batch_size=8
model_history=vgg19model.fit_generator(
        train_generator,
        steps_per_epoch=6000//batch_size,#number of pictures in training data set divided by the batch size
        epochs=10,
        validation_data=val_generator,
        validation_steps= 1000// batch_size,#number of pictures in validation data set divided by the batch size
        callbacks=callbacks_list)

<ipython-input-46-676e48da7cfc>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_history=vgg19model.fit_generator(


Epoch 1/10
750/750 [==============================] - ETA: 0s - loss: 1.6809 - acc: 0.4269
Epoch 1: loss improved from 2.15913 to 1.68094, saving model to /content/drive/MyDrive/Thesis/Saved_model.h5


750/750 [==============================] - 169s 225ms/step - loss: 1.6809 - acc: 0.4269
Epoch 2/10
750/750 [==============================] - ETA: 0s - loss: 1.1657 - acc: 0.6138
Epoch 2: loss improved from 1.68094 to 1.16566, saving model to /content/drive/MyDrive/Thesis/Saved_model.h5


750/750 [==============================] - 175s 233ms/step - loss: 1.1657 - acc: 0.6138
Epoch 3/10
750/750 [==============================] - ETA: 0s - loss: 0.8398 - acc: 0.7257
Epoch 3: loss improved from 1.16566 to 0.83981, saving model to /content/drive/MyDrive/Thesis/Saved_model.h5


750/750 [==============================] - 176s 234ms/step - loss: 0.8398 - acc: 0.7257
Epoch 4/10
750/750 [==============================] - ETA: 0s - loss: 0.6606 - acc: 0.7808
Epoch 4: loss improved from 0.83981 to 0.66065, saving model to /content/drive/MyDrive/Thesis/Saved_model.h5


...

750/750 [==============================] - 175s 233ms/step - loss: 0.4454 - acc: 0.8532
Epoch 7/10
750/750 [==============================] - ETA: 0s - loss: 0.3786 - acc: 0.8765
Epoch 7: loss improved from 0.44539 to 0.37860, saving model to /content/drive/MyDrive/Thesis/Saved_model.h5


750/750 [==============================] - 176s 234ms/step - loss: 0.3786 - acc: 0.8765
Epoch 8/10
750/750 [==============================] - ETA: 0s - loss: 0.3515 - acc: 0.8864
Epoch 8: loss improved from 0.37860 to 0.35147, saving model to /content/drive/MyDrive/Thesis/Saved_model.h5


750/750 [==============================] - 174s 232ms/step - loss: 0.3515 - acc: 0.8864
Epoch 9/10
750/750 [==============================] - ETA: 0s - loss: 0.3052 - acc: 0.8991
Epoch 9: loss improved from 0.35147 to 0.30522, saving model to /content/drive/MyDrive/Thesis/Saved_model.h5


750/750 [==============================] - 176s 234ms/step - loss: 0.3052 - acc: 0.8991
Epoch 10/10
750/750 [==============================] - ETA: 0s - loss: 0.2843 - acc: 0.9039
Epoch 10: loss improved from 0.30522 to 0.28425, saving model to /content/drive/MyDrive/Thesis/Saved_model.h5


750/750 [==============================] - 175s 233ms/step - loss: 0.2843 - acc: 0.9039


In [2]:
#size of the plots
fig=plt.figure(figsize=(15,5))
columns = 2
rows = 1

#plot loss
#the accuracy and loss are stored in the "model_history"
fig.add_subplot(rows, columns, 1)
plt.plot(model_history.history['loss']) #merge the loss from the two training steps
plt.plot(model_history.history['loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

#plot accuracy
fig.add_subplot(rows, columns, 2)
plt.plot(model_history.history['acc'])
plt.plot(model_history.history['acc'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: ignored

In [13]:
model=load_model("/content/drive/MyDrive/Thesis/Saved_model.h5")



In [14]:
#load the testing images
#As in the training and validation datasets, the testing pictures folder should be organized in
#different sub-folders with the pictures of each individual in a different sub-folder.
#Note that a sub-folder for each of the individuals used in the training dataset should be present
#even if there are no pictures for some individuals.
import numpy as np

val_sony_datagen1 = ImageDataGenerator(rescale = 1./255)
val_sony_datagen = val_sony_datagen1.flow_from_directory(
        directory=r"/content/drive/MyDrive/Thesis/Detectron_cropped_images/IndividualID/great_tits/Test/", #This folder should contain pictures of each bird in a different subfolder (similar to the training data set)
        target_size=(224, 224),
        batch_size=70,#number of images in the testing dataset
        shuffle=False)

#load the pictures in the testing folder. The x_batch contains the pictures and the y_batch contains the
#identities of the individuals
x_batch, y_batch=next(val_sony_datagen)

#create lists to store the right and the wrong classifications
right_classification=[]
wrong_classification=[]
#loop through all the testing pictures and predict the identity of the individuals
for i in range(0,len(x_batch)):
    image=np.expand_dims(x_batch[i], axis=0)
    result=model.predict(image)
    #if the predicted identity matches the real identity (from the y_batch) store the
    #index of this pcitures in the right classification list. If different store it
    #in the wrong classification list
    if np.where(y_batch[i] == np.amax(y_batch[i]))[0][0]==np.where(result == np.amax(result))[1][0]:
        right_classification.append(i)
    else:
        wrong_classification.append(i)
#print the results
print(len(right_classification)/(len(wrong_classification)+len(right_classification)))

Found 63 images belonging to 10 classes.
1/1 [==============================] - 0s 22ms/step
0.746031746031746
